In [ ]:
#pip install lazypredict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("../input/heart-attack-analysis-prediction-dataset/heart.csv")
heart = df.copy()
heart

In [ ]:
heart.info()

In [ ]:
heart.describe()

In [ ]:
heart.output.value_counts()

In [ ]:
heart[heart.duplicated()]

In [ ]:
heart.drop_duplicates(inplace=True)
heart.shape

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(heart.corr(), annot=True, cmap="coolwarm")
plt.show()

In [ ]:
X = heart.drop(["output"], axis=1)
y = heart["output"]

In [ ]:
cat_cols = []
num_cols = []

for col in X.columns:
    if(X[col].nunique() <= 5):
        cat_cols.append(col)
    else:
        num_cols.append(col)
print(f"categorical:{cat_cols}\n"
      f"Numerical:{num_cols}")

In [ ]:
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train[num_cols] = scalar.fit_transform(X_train[num_cols])
X_test[num_cols] = scalar.transform(X_test[num_cols])

In [ ]:
#lazypredict couldnot run on note
"""
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier()
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(predictions)
"""

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

svc_param = {"C":[100, 1000], "kernel":["linear", "rbf"], "degree":list(map(int, range(6)))}
knn_param = {"n_neighbors":list(map(int, range(11))), "weights":["uniform", "distance"]}
lr_param = {"C":list(map(int, range(11)))}
#mlp_param = {"hidden_layer_sizes":[100, 200, 300, 400, 500], "learning_rate":["constant", "adaptive"], "warm_start":[True, False]}
mlp_param = {"learning_rate":["constant", "adaptive"], "warm_start":["True", "False"], "solver":["sgd", "adam"]}

name_list = ["LR", "KNN", "SVC", "DNN"]
model_list = [LogisticRegression(), KNeighborsClassifier(), SVC(), MLPClassifier()]
param_list = [lr_param, knn_param, svc_param, mlp_param]

for name, model, param in zip(name_list, model_list, param_list):
    clf = GridSearchCV(model, param).fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(f"{name}:\n\t{clf.best_params_}\n\t{clf.best_score_:.3f}")

In [ ]:
model1 = LogisticRegression(C=3).fit(X_train, y_train)
model2 = KNeighborsClassifier(n_neighbors=8, weights="distance").fit(X_train, y_train)
model3 = SVC(C=100 ,kernel="linear" ,degree=0).fit(X_train, y_train)
model4 = MLPClassifier(learning_rate="constant", warm_start=False, solver="adam", random_state=0).fit(X_train, y_train)

name_list = ["LR", "KNN", "SVC", "DNN"]
model_list = [model1, model2, model3, model4]

X[num_cols] = scalar.transform(X[num_cols])

for name, model in zip(name_list, model_list):
    cv_score = cross_val_score(model, X_train, y_train, cv=10)
    print(f"{name}:\n\tCV10_Accuracy:{cv_score.mean():5>.3f}\n\tTest_Accuracy:{accuracy_score(model.predict(X_test), y_test):5>.3f}")